# Lab | Customer Analysis Round 5

For this lab, we still keep using the `marketing_customer_analysis.csv` file that you can find in the `files_for_lab` folder.

### Get the data

We are using the `marketing_customer_analysis.csv` file.

### Dealing with the data

Already done in the round 2.

### Explore the data

Done in the round 3.

### Processing Data

(_Further processing..._)

- X-y split.
- Normalize (numerical).


In [41]:
# Importing the libraries

import pandas as pd

import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [42]:
# Importing the dataframe

customer_df = pd.read_csv("/Users/leozinho.air/Desktop/ironhack_da/class_04/lab-customer-analysis-round-2/files_for_lab/csv_files/marketing_customer_analysis.csv")

# Cleaning operations

columns = []

for i in range(len(customer_df.columns)):
    columns.append(customer_df.columns[i].lower().replace(' ', '_'))

customer_df.columns = columns

customer_df

# Dropping the column 'unnamed:_0'

customer_df = customer_df.drop(['unnamed:_0'], axis = 1)

# The only values are Nan or vehicle class 'A', let's create a new df without the column.

customer_df = customer_df.drop(['vehicle_type'], axis = 1)

customer_df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,...,52,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize
1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,...,26,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,22139,...,31,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize
3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,49078,...,3,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,23675,...,31,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,0,...,40,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize
10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,61146,...,68,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize
10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,39837,...,63,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize
10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,64195,...,27,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize


In [43]:
# This is for predictions
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# These Libs are for stats -> this ones are for description
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Creating a dataframe with only numerical values

num_cust = customer_df.select_dtypes(include = np.number)

# Checking for null values

num_cust.isnull().any() # Two columns have nan values

# I chose to fill the nan valus with the means of the two columns

mean_months = num_cust['months_since_last_claim'].mean()

num_cust['months_since_last_claim'].fillna(value = mean_months, inplace = True)


mean_complaints = num_cust['number_of_open_complaints'].mean()
num_cust['number_of_open_complaints'].fillna(value = mean_complaints, inplace = True)

num_cust.isnull().sum().sum() # There are no nan values




0

In [44]:
# X-y split.

Y = customer_df['total_claim_amount']
X = customer_df.drop(['total_claim_amount'], axis = 1)

# Normalizing the DataFrame

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() # This is the normalization process

normalized_data = scaler.fit_transform(num_cust) # Fit it to the data
normalized_data = pd.DataFrame(normalized_data, columns = num_cust.columns)

normalized_data

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0.035752,0.480381,0.000000,0.200000,0.525253,0.000000,1.000,0.101171
1,0.004059,0.000000,0.012658,0.085714,0.262626,0.000000,0.000,0.257445
2,0.160264,0.221432,0.164557,0.971429,0.313131,0.000000,0.125,0.165875
3,0.250953,0.490873,0.151899,0.285714,0.030303,0.000000,0.125,0.167263
4,0.087527,0.236795,0.236287,0.432831,0.313131,0.076851,0.750,0.244657
...,...,...,...,...,...,...,...,...
10905,0.167823,0.000000,0.810127,0.432831,0.404040,0.076851,0.750,0.419717
10906,0.041281,0.611576,0.016878,0.200000,0.686869,0.000000,0.625,0.094333
10907,0.270122,0.398446,0.590717,0.314286,0.636364,0.000000,0.125,0.131763
10908,0.123717,0.642072,0.409283,0.000000,0.272727,0.800000,0.625,0.213674
